In [10]:
import uuid

import pprint
import logging
import boto3

logging.basicConfig(format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)

In [16]:
input_text:str = "I am looking to book a flight from Calgary to Paris on Apr 1 and my membership number is 123"
agent_id:str = 'ZUCW23RWFO'                                             # note this from the agent console on Bedrock
agent_alias_id:str = 'TSTALIASID'                                       # fixed for draft version of the agent
session_id:str = str(uuid.uuid1())                                      # random identifier
enable_trace:bool = True

In [17]:
session = boto3.Session(profile_name='bach-dev', region_name='us-east-1')
client = session.client(service_name="bedrock-agent-runtime")

logger.info(client)

[2024-03-20 15:26:40,964] p1168 {276264083.py:4} INFO - <botocore.client.AgentsforBedrockRuntime object at 0x000001DB50028760>


In [18]:
# invoke the agent API
response = client.invoke_agent(inputText=input_text,
    agentId=agent_id,
    agentAliasId=agent_alias_id,
    sessionId=session_id,
    enableTrace=enable_trace
)

logger.info(pprint.pprint(response))

[2024-03-20 15:26:42,691] p1168 {1436172374.py:9} INFO - None


{'ResponseMetadata': {'HTTPHeaders': {'connection': 'keep-alive',
                                      'content-type': 'application/json',
                                      'date': 'Wed, 20 Mar 2024 21:26:42 GMT',
                                      'transfer-encoding': 'chunked',
                                      'x-amz-bedrock-agent-session-id': '89994381-e700-11ee-976e-98e743dac758',
                                      'x-amzn-bedrock-agent-content-type': 'application/json',
                                      'x-amzn-requestid': '73617490-9b97-4797-9bf2-41c9b33bdb27'},
                      'HTTPStatusCode': 200,
                      'RequestId': '73617490-9b97-4797-9bf2-41c9b33bdb27',
                      'RetryAttempts': 0},
 'completion': <botocore.eventstream.EventStream object at 0x000001DB504DF430>,
 'contentType': 'application/json',
 'sessionId': '89994381-e700-11ee-976e-98e743dac758'}


In [19]:
%%time
import json
event_stream = response['completion']
try:
    for event in event_stream:        
        if 'chunk' in event:
            data = event['chunk']['bytes']
            logger.info(f"Final answer ->\n{data.decode('utf8')}") 
            end_event_received = True
            # End event indicates that the request finished successfully
        elif 'trace' in event:
            logger.info(json.dumps(event['trace'], indent=2))
        else:
            raise Exception("unexpected event.", event)
except Exception as e:
    raise Exception("unexpected event.", e)


[2024-03-20 15:26:44,450] p1168 {<timed exec>:11} INFO - {
  "agentId": "ZUCW23RWFO",
  "agentAliasId": "TSTALIASID",
  "sessionId": "89994381-e700-11ee-976e-98e743dac758",
  "trace": {
    "preProcessingTrace": {
      "modelInvocationInput": {
        "traceId": "73617490-9b97-4797-9bf2-41c9b33bdb27-pre-0",
        "text": "You are a classifying agent that filters user inputs into categories. Your job is to sort these inputs before they are passed along to our function calling agent. The purpose of our function calling agent is to call functions in order to answer user's questions.\n\nHere is the list of functions we are providing to our function calling agent. The agent is not allowed to call any other functions beside the ones listed here:\n<tools>\n    <tool_description>\n<tool_name>GET::TravelAgentGroup::getAirportCode</tool_name>\n<description>Based on the provided city name, returns the airport code closest to the city</description>\n<parameters>\n<parameter>\n<name>City</name>

[2024-03-20 15:26:51,765] p1168 {<timed exec>:11} INFO - {
  "agentId": "ZUCW23RWFO",
  "agentAliasId": "TSTALIASID",
  "sessionId": "89994381-e700-11ee-976e-98e743dac758",
  "trace": {
    "preProcessingTrace": {
      "modelInvocationOutput": {
        "traceId": "73617490-9b97-4797-9bf2-41c9b33bdb27-pre-0",
        "parsedResponse": {
          "rationale": "This input from the user appears to be a request to book a flight from Calgary to Paris on a specific date, and they have provided their membership number. The input does not seem malicious or attempting to manipulate my behavior. It is a reasonable question that could potentially be answered by the function calling agent if it calls the appropriate functions. Therefore, I will categorize this input as Category D - a question that can be answered by the function calling agent using the allowed functions.",
          "isValid": true
        }
      }
    }
  }
}
[2024-03-20 15:26:51,767] p1168 {<timed exec>:11} INFO - {
  "agentI

CPU times: total: 0 ns
Wall time: 45.8 s
